In [ ]:
import os
import glob
from osgeo import osr,gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# !pip install ipython-autotime
# %load_ext autotime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
photon_df = pd.read_csv("/content/drive/MyDrive/PS data/final.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
ds = gdal.Open("/content/drive/MyDrive/PS data/NSikkim_SRTMGL1Ellip.tif")
transform = ds.GetGeoTransform()
band = ds.GetRasterBand(1)
array = band.ReadAsArray()
 
#plt.imshow(array)
array

array([[5822.1035, 5805.1045, 5795.1055, ..., 4688.8496, 4673.8496,
        4658.85  ],
       [5813.102 , 5768.103 , 5750.104 , ..., 4688.8496, 4668.85  ,
        4654.85  ],
       [5794.101 , 5744.102 , 5719.103 , ..., 4681.8496, 4664.85  ,
        4651.8506],
       ...,
       [4960.5093, 4970.5103, 4981.5117, ..., 1800.882 , 1851.8834,
        1891.885 ],
       [4968.505 , 4979.5063, 4986.508 , ..., 1779.8793, 1832.8807,
        1875.8823],
       [4962.501 , 4982.5024, 4996.504 , ..., 1777.8765, 1801.878 ,
        1862.8795]], dtype=float32)

In [ ]:
width = ds.RasterXSize
height = ds.RasterYSize
print(width)
print(height)

2321
1197


In [ ]:
gt = ds.GetGeoTransform()

In [ ]:
origin_x = gt[0]
origin_y = gt[3]
pixel_res_x = gt[1]
pixel_res_y = gt[5]

In [ ]:
def latitude(n):
    return origin_y + (n/width * pixel_res_y)
def longitude(n):
    return origin_x + (n%width * pixel_res_x)

In [ ]:
array = np.array(array)
array = array.flatten()
dem_df = pd.DataFrame(data = array,columns = ['dem height'])
dem_df['index1']=dem_df.index
dem_df['latitude'] = dem_df.index1.apply(latitude)
dem_df['longitude'] = dem_df.index1.apply(longitude)
dem_df = dem_df.drop(['index1'],axis=1)
dem_df = dem_df[['latitude','longitude','dem height']]

In [ ]:
dem_df

,latitude,longitude,dem height
0,27.874028,88.028194,5822.103516
1,27.874028,88.028472,5805.104492
2,27.874028,88.028750,5795.105469
3,27.874027,88.029028,5765.106445
4,27.874027,88.029306,5758.107422
...,...,...,...
2778232,27.541528,88.671528,1763.873535
2778233,27.541528,88.671806,1767.875000
2778234,27.541528,88.672083,1777.876465
2778235,27.541528,88.672361,1801.878052


In [ ]:
def get_latitude_index(lat):
  return int((lat-origin_y)/pixel_res_y)

def get_longitude_index(lon):
  return int((lon-origin_x)/pixel_res_x)

def get_high_res_latitude_index(lat):
  return int((lat-origin_y)/(pixel_res_y/3))

def get_high_res_longitude_index(lon):
  return int((lon-origin_x)/(pixel_res_x/3))

In [ ]:
photon_df['latitude_index']= photon_df.latitude.apply(get_latitude_index)
photon_df['longitude_index']= photon_df.longitude.apply(get_longitude_index)
photon_df['latitude_index2']= photon_df.latitude.apply(get_high_res_latitude_index)
photon_df['longitude_index2']= photon_df.longitude.apply(get_high_res_longitude_index)

In [ ]:
photon_df['container_box_id']=(photon_df['latitude_index'])*(width)+photon_df['longitude_index']

In [ ]:
photon_df['container_box_id2']=(photon_df['latitude_index2'])*(width*3)+photon_df['longitude_index2']

In [ ]:
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id,container_box_id2
index,,,,,,,,,
0,27.841415,88.454983,4309.2500,117,1536,352,4609,273093,2455585
1,27.841344,88.454975,4311.4966,117,1536,352,4609,273093,2455585
2,27.841325,88.454973,4312.4316,117,1536,353,4609,273093,2462548
3,27.841313,88.454972,4310.0664,117,1536,353,4609,273093,2462548
4,27.841287,88.454969,4311.5400,117,1536,353,4609,273093,2462548
...,...,...,...,...,...,...,...,...,...
170384,27.476968,88.414181,2974.6000,1429,1389,4288,4168,3318098,29861512
170386,27.476955,88.414180,2973.3796,1429,1389,4288,4168,3318098,29861512
170387,27.476949,88.414179,2970.8690,1429,1389,4288,4168,3318098,29861512


In [ ]:
photon_df=photon_df.groupby(['container_box_id2']).mean()
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id
container_box_id2,,,,,,,,
2437884,27.841537,88.105476,5948.792200,116,278,350,834,269514
2438062,27.841543,88.121953,5666.793253,116,337,350,1012,269573
2438449,27.841543,88.157734,5713.768833,116,466,350,1399,269702
2438496,27.841533,88.162106,5761.059586,116,482,350,1446,269718
2438852,27.841550,88.195082,6397.295459,116,600,350,1802,269836
...,...,...,...,...,...,...,...,...
29871178,27.476866,88.664467,2560.722950,1429,2290,4289,6871,3318999
29871188,27.476889,88.665360,2485.663000,1429,2293,4289,6881,3319002
29872284,27.476802,88.122096,4090.300300,1430,338,4290,1014,3319368


In [ ]:
photon_df=photon_df.drop(['latitude_index'],axis=1)
photon_df=photon_df.drop(['longitude_index'],axis=1)
photon_df=photon_df.drop(['latitude_index2'],axis=1)
photon_df=photon_df.drop(['longitude_index2'],axis=1)
photon_df.reset_index(drop=True, inplace=True)

In [ ]:
photon_df["container_box_id"] = photon_df["container_box_id"].astype(int)

In [ ]:
photon_df

,latitude,longitude,photon height,container_box_id
0,27.841537,88.105476,5948.792200,269514
1,27.841543,88.121953,5666.793253,269573
2,27.841543,88.157734,5713.768833,269702
3,27.841533,88.162106,5761.059586,269718
4,27.841550,88.195082,6397.295459,269836
...,...,...,...,...
148114,27.476866,88.664467,2560.722950,3318999
148115,27.476889,88.665360,2485.663000,3319002
148116,27.476802,88.122096,4090.300300,3319368
148117,27.476804,88.178649,3642.655300,3319571


In [ ]:
merged_df = photon_df.merge(dem_df, left_on='container_box_id', right_index = True)

In [ ]:
merged_df

,latitude_x,longitude_x,photon height,container_box_id,latitude_y,longitude_y,dem height
0,27.841537,88.105476,5948.792200,269514,27.841772,88.105417,5977.253906
1,27.841543,88.121953,5666.793253,269573,27.841765,88.121806,5681.316895
2,27.841543,88.157734,5713.768833,269702,27.841750,88.157639,5724.455078
3,27.841533,88.162106,5761.059586,269718,27.841748,88.162083,5768.472168
4,27.841550,88.195082,6397.295459,269836,27.841734,88.194861,6399.598145
...,...,...,...,...,...,...,...
122495,27.541653,88.130132,4247.735980,2776282,27.541762,88.129861,4263.985352
122526,27.541587,88.130125,4248.245733,2776282,27.541762,88.129861,4263.985352
122500,27.541648,88.157914,4505.259352,2776382,27.541750,88.157639,4514.117676
122532,27.541575,88.157906,4504.336393,2776382,27.541750,88.157639,4514.117676


In [ ]:
merged_df = merged_df.drop[]

In [ ]:
merged_df.drop(columns = ['container_box_id'], inplace = True)
X = merged_df.drop('photon height',axis=1)
y = merged_df['photon height']

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#To get the best parameters for our model
# from sklearn.model_selection import GridSearchCV

# model = DecisionTreeRegressor()

# grid_search = GridSearchCV(model,
#                   param_grid = {'max_depth': range(1, 11),
#                                 'min_samples_split': range(10, 60, 10)},
#                   cv=5,
#                   n_jobs=1,
#                   scoring='neg_mean_squared_error')

# grid_search.fit(X, y)

# print(grid_search.best_params_)
# print(-grid_search.best_score_)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=10,
                                  min_samples_split=40)
model.fit(X, y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=10,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=40,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
finalarray=np.zeros([height*3*width*3], dtype = int)
df= pd.DataFrame(columns=['zero'],data=finalarray)
df

,zero
0,0
1,0
2,0
3,0
4,0
...,...
25004128,0
25004129,0
25004130,0
25004131,0


In [ ]:
def latitude_final(n):
    return origin_y + (n/(width*3) * pixel_res_y/3.0)
def longitude_final(n):
    return origin_x + (n%(width*3) * pixel_res_x/3.0)

In [ ]:
df['index1']=df.index
df['latitude']=df.index1.apply(latitude_final)
df['longitude']=df.index1.apply(longitude_final)

In [ ]:
df=df.drop(columns=['index1','zero'],axis=1)
df['latitude_index']= df.latitude.apply(get_latitude_index)
df['longitude_index']= df.longitude.apply(get_longitude_index)
df['container_box_id']=(df['latitude_index'])*(width)+df['longitude_index']
df=df.drop(['latitude_index'],axis=1)
df=df.drop(['longitude_index'],axis=1)
df["container_box_id"] = df["container_box_id"].astype(int)

In [ ]:
final_merged_df = df.merge(dem_df, left_on='container_box_id', right_index = True)
final_merged_df.drop(columns = ['container_box_id'], inplace = True)

In [ ]:
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,27.874028,88.028194,27.874028,88.028194,5822.103516
1,27.874028,88.028287,27.874028,88.028194,5822.103516
2,27.874028,88.028380,27.874028,88.028194,5822.103516
6963,27.873935,88.028194,27.874028,88.028194,5822.103516
6964,27.873935,88.028287,27.874028,88.028194,5822.103516
...,...,...,...,...,...
24997168,27.541620,88.672731,27.541528,88.672639,1862.879517
24997169,27.541620,88.672824,27.541528,88.672639,1862.879517
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517


In [ ]:
final_merged_df.sort_index()

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,27.874028,88.028194,27.874028,88.028194,5822.103516
1,27.874028,88.028287,27.874028,88.028194,5822.103516
2,27.874028,88.028380,27.874028,88.028194,5822.103516
3,27.874028,88.028472,27.874028,88.028472,5805.104492
4,27.874028,88.028565,27.874028,88.028472,5805.104492
...,...,...,...,...,...
25004128,27.541528,88.672454,27.541528,88.672361,1801.878052
25004129,27.541528,88.672546,27.541528,88.672361,1801.878052
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517


In [ ]:
final_merged_df.head(6963)

,latitude_x,longitude_x,latitude_y,longitude_y,dem height
0,27.874028,88.028194,27.874028,88.028194,5822.103516
1,27.874028,88.028287,27.874028,88.028194,5822.103516
2,27.874028,88.028380,27.874028,88.028194,5822.103516
6963,27.873935,88.028194,27.874028,88.028194,5822.103516
6964,27.873935,88.028287,27.874028,88.028194,5822.103516
...,...,...,...,...,...
16244,27.873812,88.242824,27.873935,88.242639,5423.888184
16245,27.873812,88.242917,27.873935,88.242639,5423.888184
2320,27.873997,88.243009,27.873935,88.242917,5414.889160
2321,27.873997,88.243102,27.873935,88.242917,5414.889160


In [ ]:
X = final_merged_df

In [ ]:
predictions = model.predict(X)
final_merged_df['height']=predictions
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height,height
0,27.874028,88.028194,27.874028,88.028194,5822.103516,5817.748144
1,27.874028,88.028287,27.874028,88.028194,5822.103516,5817.748144
2,27.874028,88.028380,27.874028,88.028194,5822.103516,5817.748144
6963,27.873935,88.028194,27.874028,88.028194,5822.103516,5817.748144
6964,27.873935,88.028287,27.874028,88.028194,5822.103516,5817.748144
...,...,...,...,...,...,...
24997168,27.541620,88.672731,27.541528,88.672639,1862.879517,1880.300099
24997169,27.541620,88.672824,27.541528,88.672639,1862.879517,1880.300099
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517,1880.300099
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517,1880.300099


In [ ]:
final_merged_df

,latitude_x,longitude_x,latitude_y,longitude_y,dem height,height
0,27.874028,88.028194,27.874028,88.028194,5822.103516,5817.748144
1,27.874028,88.028287,27.874028,88.028194,5822.103516,5817.748144
2,27.874028,88.028380,27.874028,88.028194,5822.103516,5817.748144
6963,27.873935,88.028194,27.874028,88.028194,5822.103516,5817.748144
6964,27.873935,88.028287,27.874028,88.028194,5822.103516,5817.748144
...,...,...,...,...,...,...
24997168,27.541620,88.672731,27.541528,88.672639,1862.879517,1880.300099
24997169,27.541620,88.672824,27.541528,88.672639,1862.879517,1880.300099
25004130,27.541528,88.672639,27.541528,88.672639,1862.879517,1880.300099
25004131,27.541528,88.672731,27.541528,88.672639,1862.879517,1880.300099


In [ ]:
final_merged_df=final_merged_df.sort_index()

In [ ]:
final_merged_df=final_merged_df.drop(columns=['latitude_x','longitude_x','latitude_y','longitude_y','dem height'])

In [ ]:
final_merged_df

,height
0,5817.748144
1,5817.748144
2,5817.748144
3,5794.741071
4,5794.741071
...,...
25004128,1793.955026
25004129,1793.955026
25004130,1880.300099
25004131,1880.300099


In [ ]:
finalarray = final_merged_df.to_numpy()
finalarray=finalarray.reshape(height*3 ,width*3)
finalarray

array([[5817.74814408, 5817.74814408, 5817.74814408, ..., 4685.05179666,
        4685.05179666, 4685.05179666],
       [5817.74814408, 5817.74814408, 5817.74814408, ..., 4685.05179666,
        4685.05179666, 4685.05179666],
       [5817.74814408, 5817.74814408, 5817.74814408, ..., 4685.05179666,
        4685.05179666, 4685.05179666],
       ...,
       [4990.37740547, 4990.37740547, 4990.37740547, ..., 1880.30009892,
        1880.30009892, 1880.30009892],
       [4990.37740547, 4990.37740547, 4990.37740547, ..., 1880.30009892,
        1880.30009892, 1880.30009892],
       [4990.37740547, 4990.37740547, 4990.37740547, ..., 1880.30009892,
        1880.30009892, 1880.30009892]])

In [ ]:
dst_filename = 'sikkim2decisiontree2.tiff'
x_pixels = width*3  # number of pixels in x
y_pixels = height*3  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(finalarray)

0

In [ ]:
gt

(88.02819444448,
 0.00027777777777781464,
 0.0,
 27.87402777777352,
 0.0,
 -0.00027777777777781464)

In [ ]:
gtlist = list(gt)
gtlist[1]=gtlist[1]/3
gtlist[5]=gtlist[5]/3
gt=tuple(gtlist)

In [ ]:
geotrans = gt  #get GeoTranform from existed 'data0'
proj=ds.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [ ]:
ls

drive/  sample_data/  sikkim2decisiontree2.tiff  sikkim2decisiontree.tiff


In [ ]:
from google.colab import files
files.download('sikkim2decisiontree2.tiff')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>